## 2. a) Create an **_Inverted Index_** for the given documents.

In [27]:
class term:
    def __init__(self, s, freq, postings=[]):
        self.s = s
        self.freq = freq
        self.postings = postings

In [67]:
n = int(input("Enter the number of documents to read: "))

docs = []
terms = []
for i in range(n):
    with open(f"doc{i+1}.txt", "r") as f:
        docs.append(f.read())
    terms += [[j.lower(), i+1] for j in docs[i].split()]
terms = sorted(terms)

termPostings = []
i = 0
while i < len(terms):
    postings = [terms[i][1]]
    freq = 1
    j = i + 1
    while j < len(terms) and terms[i][0] == terms[j][0]:
        postings.append(terms[j][1])
        freq += 1
        j += 1
    termPostings.append(term(terms[i][0], freq, set(postings)))
    i = j
for i in range(len(terms)):
    terms[i] = terms[i][0]
terms = sorted(list(set(terms)))
for i in termPostings:
    print(i.s, i.freq, i.postings)

Enter the number of documents to read:  4


approach 1 {3}
breakthrough 1 {1}
drug 2 {1, 2}
for 3 {1, 3, 4}
hopes 1 {4}
new 3 {2, 3, 4}
of 1 {3}
patients 1 {4}
schizophrenia 4 {1, 2, 3, 4}
treatment 1 {3}


In [68]:
print(terms)

['approach', 'breakthrough', 'drug', 'for', 'hopes', 'new', 'of', 'patients', 'schizophrenia', 'treatment']


--- 

## 2. b) Process boolean queries

In [70]:
def binOp(termPostings, op, n):
    if op == 'NOT':
        return termPostings[0] - set(range(1, n+1))
    elif op == 'AND':
        return termPostings[0].intersection(termPostings[1])
    elif op == 'OR':
        return termPostings[0].union(termPostings[1])

In [80]:
def andor(query, n):
    vals = []
    flag = 0
    for i in range(len(query)):
        if query[i] not in ["AND", "OR"]:
            vals.append(query[i])
            if flag == 1:
                vals = binOp(vals, query[i-1], n)
                flag = 0
        else:
            flag = 1
    return vals

In [89]:
def processQ(query, n):
    sIdx = 0
    ans = []
    while sIdx < len(query):
        sIdx = query.index('(') if '(' in query else -1
        if sIdx == -1:
            break
        for i in range(sIdx, len(query)):
            idx1 = query.index('(', sIdx) if '(' in query else -1
            idx2 = query.index(')', sIdx) if ')' in query else -1
            if idx1 > idx2: break
            sIdx = idx1 if idx1 != -1 else sIdx
        eIdx = query.index(')', sIdx)
        ans = processQ(query[sIdx+1:eIdx], n)
        del query[sIdx+1:eIdx+1]
        query[sIdx] = ans
    idx = 0
    while idx < len(query):
        idx = query.index("NOT") if "NOT" in query else -1
        if idx == -1:
            break
        ans = binOp([query[idx+1]], "NOT", n)
        del query[idx+1]
        query[idx] = ans
    query = andor(query, n)
    return query

In [90]:
query = input("Enter the boolean query:").split()
n = int(input("Enter number of documents: "))
flag = 0
for i in range(len(query)):
    if query[i] in terms:
        query[i] = termPostings[terms.index(query[i])].postings
    elif query[i] in ["(", ")", "NOT", "AND", "OR"]:
        continue
    else:
        print(query[i])
        print("Invalid query!")
        flag = 1
        break
if flag == 0:
    ans = processQ(query, 4)
    print("The request query is processed and the documents are: ")
    for i in ans:
        print(f"Document{i}: ", docs[i-1])

Enter the boolean query: ( schizophrenia AND approach ) OR new
Enter number of documents:  4


The request query is processed and the documents are: 
Document2:  new schizophrenia drug
Document3:  new approach for treatment of schizophrenia
Document4:  new hopes for schizophrenia patients
